In [ ]:
!pip install -U -q PyDrive

In [ ]:
from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense

In [ ]:
# !python -m spacy download en_core_web_md

2023-05-12 22:02:26.566581: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 22.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
# import spacy
# from sklearn.metrics.pairwise import cosine_similarity

# # Download and load the pre-trained model
# nlp = spacy.load("en_core_web_md")

In [ ]:
with open('/content/drive/MyDrive/RP Work/website.json') as file:
    data = json.load(file)
training_sentences = []
training_labels = []
labels = []
responses = []
for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
num_classes = len(labels)
len(training_sentences)

418

In [ ]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)
training_labels

array([38, 38, 38, 38, 38, 37, 37, 37, 93, 93, 93, 93,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 90, 90, 90, 90, 87, 87, 87,
       87, 87, 87, 87, 87, 87, 87, 87, 89, 89, 89, 89, 89, 89, 89, 89, 89,
       89, 91, 91, 91, 91, 91, 91, 91, 91, 91, 91, 91, 91, 91, 83, 83, 83,
       83, 83, 85, 85, 85, 85, 88, 88, 88, 88, 86, 86, 86, 86, 86, 86, 86,
       86, 82, 82, 82, 82, 84, 84, 84, 84, 84, 84, 92, 92, 92, 92, 79, 79,
       79, 79, 76, 76, 76, 76, 76, 76, 80, 80, 80, 80, 80, 80, 80, 80, 80,
       80, 80, 80, 80, 72, 72, 72, 72, 72, 78, 78, 78, 78, 78, 78, 78, 78,
       78, 78, 74, 74, 74, 74, 77, 77, 77, 77, 75, 75, 75, 75, 75, 75, 75,
       75, 71, 71, 71, 71, 73, 73, 73, 73, 73, 73, 81, 81, 81, 81, 68, 68,
       68, 68, 65, 65, 65, 65, 65, 65, 69, 69, 69, 69, 69, 69, 69, 69, 69,
       69, 69, 69, 69, 61, 61, 61, 61, 61, 67, 67, 67, 67, 67, 67, 67, 67,
       67, 67, 63, 63, 63, 63, 66, 66, 66, 66, 64, 64, 64, 64, 64, 64, 64,
       64, 60, 60, 60, 62

In [ ]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

# padded_sequences

# vocab_size = 1000
# embedding_dim = 16
# max_len = 300
# oov_token = "<OOV>"
# vector_sequence=[]
# for sentence in training_sentences:
#   vector_sequence.append(nlp(sentence).vector)
# padded_sequences = pad_sequences(vector_sequence, truncating='post', maxlen=max_len)
# len(vector_sequence[0])

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2,return_sequences=True)))
model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 16)            16000     
                                                                 
 bidirectional (Bidirectiona  (None, 20, 256)          148480    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              394240    
 nal)                                                            
                                                                 
 dense (Dense)               (None, 32)                8224      
                                                                 
 dense_1 (Dense)             (None, 94)                3102      
                                                                 
Total params: 570,046
Trainable params: 570,046
Non-tr

In [ ]:
epochs = 200
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs,batch_size=30)

Epoch 1/200
14/14 [==============================] - 18s 265ms/step - loss: 4.5347 - accuracy: 0.0191
Epoch 2/200
14/14 [==============================] - 4s 255ms/step - loss: 4.4856 - accuracy: 0.0191
Epoch 3/200
14/14 [==============================] - 4s 275ms/step - loss: 4.4370 - accuracy: 0.0263
Epoch 4/200
14/14 [==============================] - 6s 388ms/step - loss: 4.3612 - accuracy: 0.0239
Epoch 5/200
14/14 [==============================] - 4s 262ms/step - loss: 4.1940 - accuracy: 0.0502
Epoch 6/200
14/14 [==============================] - 4s 253ms/step - loss: 4.0446 - accuracy: 0.0622
Epoch 7/200
14/14 [==============================] - 12s 874ms/step - loss: 3.9145 - accuracy: 0.0670
Epoch 8/200
14/14 [==============================] - 4s 294ms/step - loss: 3.8285 - accuracy: 0.0718
Epoch 9/200
14/14 [==============================] - 5s 382ms/step - loss: 3.7208 - accuracy: 0.0718
Epoch 10/200
14/14 [==============================] - 4s 298ms/step - loss: 3.6340 - accu

KeyboardInterrupt: ignored

In [ ]:
model.save("/content/drive/MyDrive/RP Work/blstm.h5")

import pickle

# to save the fitted tokenizer
with open('/content/drive/MyDrive/RP Work/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('/content/drive/MyDrive/RP Work/label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# pip install colorama

In [ ]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder


import random
import pickle

with open("/content/drive/MyDrive/RP Work/website.json") as file:
    data = json.load(file)


def chat():
    model = keras.models.load_model('/content/drive/MyDrive/RP Work/blstm.h5')
    with open('/content/drive/MyDrive/RP Work/tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
    with open('/content/drive/MyDrive/RP Work/label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)
    max_len = 20
    while True:
        print("User: ", end="")
        inp = input()
        if inp.lower() == "quit":
            break
        
        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        # print(result,end="")
        # print(np.argmax(result))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])
        # print(tag,end="")
        for i in data['intents']:
            if i['tag'] == tag:
                print("ChatBot:", np.random.choice(i['responses']))

        


In [ ]:
chat()